In [3]:
%pip install scikit-image
%pip install keras
%pip install tensorflow --upgrade --force-reinstall
%pip install tensorflow as tf
%pip install --upgrade Pillow



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tensorflow-2.5.0-cp36-cp36m-manylinux2010_x86_64.whl (454.3 MB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached h5py-3.1.0-cp36-cp36m-manylinux1_x86_64.whl (4.0 MB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-3.17.3-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached grpcio-1.34.1-cp36-cp36m-manylinux2014_x86_64.whl (4.0 MB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 

In [4]:
import keras
import tensorflow
from skimage import io
import os
import glob
import numpy as np
import cv2
# import matplotlib.pyplot as plt
from PIL import Image
# %matplotlib inline
PATH = 'Data'
image_path = os.path.join(PATH, 'Input', '*')
image_paths = sorted(glob.glob(image_path))

type(image_paths)

list

In [44]:
# load images as Numpy arrays

#images = io.imread_collection(os.path.join(PATH, 'Input', '*.jpg'))

In [ ]:
#max_height=(max(image.shape[0] for image in images))
#max_width=(max(image.shape[1] for image in images))

#min_height=(min(image.shape[0] for image in images))
#min_width=(min(image.shape[1] for image in images))

#print("Max H,W:", max_height, max_width, "Mins H,W:", min_height, min_width)

## Option A) Padding Image to Largest Size

In [2]:

height=4032
width=5312
#image_num = len(img_normalized)
for file in image_paths:
    img = cv2.imread(file, cv2.IMREAD_COLOR)
    h, w = img.shape[:2]
    pad_bottom, pad_right = 0, 0
    pad_bottom = abs(height - h)
    pad_right = abs(width - w)

    padded_image = cv2.copyMakeBorder(img,0,pad_bottom,0,pad_right,borderType=cv2.BORDER_CONSTANT,value=[0,0,0])
    cv2.imwrite(os.path.join('Data/Padded', os.path.basename(file)), padded_image) 

## Option B) Resizing Images to Smallest Size

In [3]:
#Try resizing of images
#Use Min Height and Width to set a standard input dimension for CNN architecture (ie. 128 * 128)

#Use nearest neighbor interpolation technique. If you resize your image using any other interpolation (for segmentation), it may result in tampering with the ground truth labels.
#Note: this does not preserve aspect ratio and could pose issues if we have a large difference in ratios (ie. distorting images)
new_width, new_height = 270, 180

for file in image_paths:
    img = cv2.imread(file, cv2.IMREAD_COLOR)
    resized_image = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(os.path.join('Data/Resized', os.path.basename(file)), resized_image) 


In [4]:
padded_path= os.path.join(PATH, 'Padded', '*')
padded_paths = sorted(glob.glob(padded_path))

In [ ]:
for file in padded_paths:
    img = cv2.imread(file, cv2.IMREAD_COLOR)
    norm_image=cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    cv2.imwrite(os.path.join('Data/Normalized_Padded', os.path.basename(file)), norm_image) 

In [3]:
resized_path= os.path.join(PATH, 'Resized', '*')
resized_paths = sorted(glob.glob(resized_path))

In [4]:
for file in resized_paths:
    img = cv2.imread(file, cv2.IMREAD_COLOR)
    norm_image=cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32FC1)
    cv2.imwrite(os.path.join('Data/Normalized_Resized', os.path.basename(file)), norm_image) 

## Output

In [5]:
output_path = os.path.join(PATH, 'Output', '*')
output_paths = sorted(glob.glob(output_path))
output_paths[0]

'Data/Output/003a1a43-509b-4ce7-922b-2a6851e4ef41_1296e748-81af-4fb4-b6e6-99be1c6b8a11.txt'

In [6]:
output_num = len(output_paths)
input_num = len(image_paths)
if output_num != input_num:
    print('WARNING: Number of output files does not match the number of images')
print(f'Total number of output files: {output_num}')
print(f'Total number of input files: {input_num}')

Total number of output files: 964
Total number of input files: 964


In [7]:
# Getting the target vector
y_target = []
classes = [0, 1, 2, 3]
for path in output_paths:
    boxes = open(path).read().strip().split('\n')
    max_class = 0
    for box in boxes:
        box_type = box.split(' ')[0]
        if box_type.isdigit():
            if int(box_type) not in classes:
                print(f'Unknown class: {box_type} at path {path}')
            else:
                max_class = max(max_class, int(box_type))
    y_target.append(path[12:-2] + ".jpeg")
    y_target.append(max_class)

y_target = np.array(y_target)
print('size of y_target', y_target.shape)
y_target[:10]

size of y_target (1928,)


array(['003a1a43-509b-4ce7-922b-2a6851e4ef41_1296e748-81af-4fb4-b6e6-99be1c6b8a11.t.jpeg',
       '2',
       '003d0583-7137-402d-82a4-de182b2287af_7e6dee91-df3a-44b1-8ab2-2980ad5985b6.t.jpeg',
       '1',
       '004ac576-d124-4a75-9662-d0cde84c0447_44cc47bc-6d23-49d6-a528-325727f5019d.t.jpeg',
       '2',
       '005cf1ba-0946-438f-a8d9-ff5d4b4cb70a_9b9e7c56-8c7e-4ff9-80a6-366c6945c8b5.t.jpeg',
       '1',
       '005ede95-b5d6-4b58-b7bd-fdba8e28eb86_604b9d8c-18ba-4d9c-aa7e-abe07dca58b9.t.jpeg',
       '1'], dtype='<U83')

In [8]:
# classification_target = (y_target!=0).astype(np.int)
# classification_target[:10]

In [9]:
# Save y_target to file (Data/Target/y_target.txt)
y_target_path = os.path.join(PATH, 'Target', 'y_target_with_name.txt')
txtfile = open(y_target_path, "w")
for element in y_target:
    txtfile.write(element + "\n")
txtfile.close()

In [ ]:
# Get y_target from file
y_target_from_file = np.loadtxt(y_target_path, dtype=int)
print(type(y_target))
print(type(y_target_from_file))
y_target_from_file[:10]

In [12]:
# Getting the target vector from /root/train/labels
import os, glob
import numpy as np
output_path = os.path.join('/root/train/labels', '*')
output_paths = sorted(glob.glob(output_path))
print(output_paths[0])

y_target = []
classes = [0, 1, 2, 3]
for path in output_paths:
    boxes = open(path).read().strip().split('\n')
    max_class = 0
    for box in boxes:
        box_type = box.split(' ')[0]
        if box_type.isdigit():
            if int(box_type) not in classes:
                print(f'Unknown class: {box_type} at path {path}')
            else:
                max_class = max(max_class, int(box_type))
    y_target.append(max_class)

y_target = np.array(y_target)
print('size of y_target', y_target.shape)
y_target[:10]

/root/train/labels/003a1a43-509b-4ce7-922b-2a6851e4ef41_1296e748-81af-4fb4-b6e6-99be1c6b8a11_jpg.rf.71d233698256113e8d00edba7664553a.txt
size of y_target (670,)


In [19]:
np.savetxt('y_target_train', y_target, fmt='%d')